# Benchmarking models with static data

In [2]:
import sys
sys.path.append('/home/sedroc/Bachelor/FEPCEU')

In [3]:
import numpy as np
import pandas as pd

from models.baseline import Baseline
from models.arima import Arima
from models.lgbm import LGBM
from models.chronos import Chronos
from experiment import TimeSeriesExperiment
from data.data_retriever import Dataretreiver

/home/sedroc/Bachelor/FEPCEU/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# Create sample DataFrame
data = Dataretreiver()
df = data.combined

In [6]:
# Define models
models = {
    'BASELINE': Baseline(),
    #'SARIMAX': Arima(order=[1,1,1], seasonal_order=[1,1,1,24]),
    'LGBM': LGBM()
    #'Chronos': Chronos()
}

In [7]:
# Initialize the experiment
experiment = TimeSeriesExperiment(
    models=models,
    target_column='price',
    forecast_horizon=168,
    n_splits=1,
)

# Run the experiment with custom start date
experiment.run_experiment(
    df, 
    exog_columns=['sun', 'wind', 'temp'],
    first_split_date='2024-10-01'  # Start CV from this date
)

# Get the results
results_df = experiment.summarize_results()
print(results_df)

Cross-validation splits:
Split 1: Train end = 2024-10-01 00:00:00 (index 6576), Test end = 2024-10-07 23:00:00 (index 6743)
Running CV split 1/1: Train end=2024-10-01 00:00:00, Test end=2024-10-07 23:00:00
len(y)=6576


Best trial: 12. Best value: 203.254: 100%|██████████| 20/20 [01:46<00:00,  5.31s/it]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 700, 'max_depth': 9, 'min_data_in_leaf': 239, 'learning_rate': 0.3791733226905797, 'feature_fraction': 0.7631871251516885, 'max_bin': 248, 'reg_alpha': 0.9615785382997593, 'reg_lambda': 0.2651957480262184}
  Backtesting metric: 203.25387361900164
               MSE_mean  MSE_std    MAE_mean  MAE_std  MAPE_mean  MAPE_std  \
BASELINE  161739.119378      0.0  345.767323      0.0   4.706244       0.0   
LGBM       80767.781026      0.0  230.427962      0.0   7.410776       0.0   

           R2_mean  R2_std  
BASELINE -1.555050     0.0  
LGBM     -0.275917     0.0  


In [8]:
# Plot MAE results with error bars
experiment.plot_cv_results('MAE')

In [9]:
# Plot predictions
experiment.plot_predictions(df)